In [31]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.1.0


In [30]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13383009489541629199
]


In [20]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            return pd.read_csv(path+file, index_col=False)

In [21]:
# import data

nBodies = 3
workDir = "C:/Users/llave/Documents/nBody/"
dataPath = workDir + "/data/"
        
df = grab_data(True, dataPath)
df.head()

,finalFile,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3,yf1,yf2,yf3,dxf1,dxf2,dxf3,dyf1,dyf2,dyf3
0,1,10001,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.442821,-3.888589,-4.899092,-2.849904,0.457428,-0.525155,0.687588,-0.222422,0.961199,-0.100621
1,1,10002,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.445500,-3.889573,-4.895165,-2.850298,0.389737,-0.420452,0.683916,-0.281080,1.049364,-0.101038
2,1,10003,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.448164,-3.890785,-4.890894,-2.850693,0.321736,-0.315285,0.680246,-0.339542,1.137235,-0.101454
3,1,10004,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.450814,-3.892225,-4.886281,-2.851090,0.253345,-0.209536,0.676576,-0.397869,1.224904,-0.101870
4,1,10005,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.453450,-3.893893,-4.881325,-2.851489,0.184487,-0.103088,0.672908,-0.456125,1.312466,-0.102286


In [22]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4

i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df[i_col].values
y1 = df[o_col].values

X_train,X_test,y_train,y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1147503, 16) (1147503, 12)
(286876, 16) (286876, 12)


In [23]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [12]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = X_train.shape[0]
num_test_examples = X_test.shape[0]

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [14]:
with strategy.scope():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(y_train.shape[1],activation='linear')
    ])

    loss_fn = tf.keras.losses.MeanSquaredError()
    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['mse'])

RuntimeError: /job:localhost/replica:0/task:0/device:GPU:0 unknown device.

In [15]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))
        
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [16]:
model.fit(train_ds, epochs=5, callbacks=callbacks)

Train for 8965 steps
Epoch 1/5
2145/8965 [======>.......................] - ETA: 23s - loss: 3.4980 - mse: 3.4980
Learning rate for epoch 1 is 0.0010000000474974513
2148/8965 [======>.......................] - ETA: 23s - loss: 3.4981 - mse: 3.4981

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test,  y_test, verbose=2)